# Import software libraries

In [ ]:
import sys                                                  # Read system parameters.
import numpy as np                                          # Work with multi-dimensional arrays.
import pandas as pd                                         # Manipulate and analyze data.
import matplotlib                                           # Create and format charts.
import matplotlib.pyplot as plt
import seaborn as sns                                       # Make charting easier.
import sklearn                                              # Train and evaluate machine learning models.
from sklearn.model_selection import train_test_split, \
                                    learning_curve, \
                                    GridSearchCV
from sklearn.linear_model import LinearRegression, \
                                 ElasticNet
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, \
                            mean_absolute_error, \
                            mean_squared_error
from sklearn.dummy import DummyRegressor
import xgboost                                              # Build gradient boosting models.
from xgboost import XGBRegressor
import pickle                                               # Save Python objects as binary files.
import warnings                                             # Suppress warnings.
warnings.filterwarnings('ignore')

# Ensure results are reproducible.
np.random.seed(1)

# Summarize software libraries used.
print('Libraries used in this project:')
print('- Python {}'.format(sys.version))
print('- NumPy {}'.format(np.__version__))
print('- pandas {}'.format(pd.__version__))
print('- Matplotlib {}'.format(matplotlib.__version__))
print('- Seaborn {}'.format(sns.__version__))
print('- scikit-learn {}'.format(sklearn.__version__))
print('- XGBoost {}'.format(xgboost.__version__))

# Load and preview the data

In [ ]:
users_data = pd.read_pickle('/home/student/CDSP/Regression/data/users_data_final.pickle')

users_data.head(n = 5)

# Check the shape of the data

In [ ]:
users_data.shape

# Check the data types

In [ ]:
users_data.info()

# Explore the distribution of the target variable

In [ ]:
users_data.total_amount_usd.describe()

In [ ]:
users_data.total_amount_usd.hist()
plt.title('Distribution of total_amount_usd');

In [ ]:
sns.boxplot(users_data.total_amount_usd, orient = 'h');

# Identify and remove the outliers

In [ ]:
q1 = np.percentile(users_data.total_amount_usd, 25)
q3 = np.percentile(users_data.total_amount_usd, 75)
iqr = q3 - q1

lb = q1 - 1.5 * iqr
ub = q3 + 1.5 * iqr

print('Lower bound:', round(lb, 2))
print('Upper bound:', round(ub, 2))

In [ ]:
print('Number of users with total_amount_usd greater than UB:',
      users_data[(users_data.total_amount_usd >= ub)].shape[0])
print('Number of users with total_amount_usd lower than LB:  ',
      users_data[(users_data.total_amount_usd <= lb)].shape[0])

In [ ]:
users_data_wout_outliers = \
users_data[(users_data.total_amount_usd < ub) \
           & (users_data.total_amount_usd > lb)]

users_data_wout_outliers.shape

# Explore the dataset with the outliers removed

In [ ]:
users_data_wout_outliers.describe()

In [ ]:
plt.title('Distribution of total_amount_usd')
users_data_wout_outliers.total_amount_usd.hist();

# Split the data into target and features

In [ ]:
target_data = users_data_wout_outliers.total_amount_usd 
features = users_data_wout_outliers.drop(['user_id', 'total_amount_usd'],
                                         axis = 1)

# Split the data into train and test sets

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, 
                                                    target_data,
                                                    test_size = 0.3)

In [ ]:
print('Training-data features: ', X_train.shape)
print('Training-data target:   ', y_train.shape)

# Check the distribution of the test data

In [ ]:
print('Test-data features: ', X_test.shape)
print('Test-data target:   ', y_test.shape)

In [ ]:
y_test.describe()

# Train a linear regression model

In [ ]:
linreg = LinearRegression()
linreg.fit(X_train, y_train)

# Make predictions using the linear regression model

In [ ]:
linreg_y_pred = linreg.predict(X_test) 

In [ ]:
results = pd.concat([y_test.iloc[:5], X_test.iloc[:5]], axis = 1)
results.insert(1, 'total_pred', linreg_y_pred[:5].round(2))
results

# Obtain the linear regression model's score

In [ ]:
r2_score(y_test, linreg_y_pred)

# Train a decision-tree model

In [ ]:
reg_tree = DecisionTreeRegressor()
reg_tree.fit(X_train, y_train)

# Make predictions using the decision-tree model

In [ ]:
reg_tree_y_pred = reg_tree.predict(X_test)

In [ ]:
results['total_pred'] = reg_tree_y_pred[:5]
results

# Obtain the decision-tree model's score

In [ ]:
r2_score(y_test, reg_tree_y_pred)

# Visualize the decision tree

In [ ]:
fig = plt.figure(figsize = (25, 20))
_ = tree.plot_tree(reg_tree, 
                   feature_names = list(X_train.columns),
                   max_depth = 2,
                   filled = True)

# Train a random-forest model

In [ ]:
rf = RandomForestRegressor()
rf.fit(X_train, y_train)

# Make predictions using the random-forest model

In [ ]:
rf_y_pred = rf.predict(X_test)

In [ ]:
results['total_pred'] = rf_y_pred[:5]
results

# Obtain the random-forest model's score

In [ ]:
r2_score(y_test, rf_y_pred)

# Train a gradient-boosting model

In [ ]:
xgb = XGBRegressor()
xgb.fit(X_train, y_train)

# Make predictions using the gradient-boosting model

In [ ]:
xgb_y_pred = xgb.predict(X_test)

In [ ]:
results['total_pred'] = xgb_y_pred[:5]
results

# Obtain the gradient-boosting model's score

In [ ]:
r2_score(y_test, xgb_y_pred)

# Define the parameter grid used to tune the linear regression model

In [ ]:
param_grid = {'l1_ratio': [0.1, 0.5, 0.9],
              'alpha': [0.0001, 0.01, 0.1],
              'max_iter': [100, 1000, 10000]}

print(param_grid)

# Perform a grid search for optimal elastic net hyperparameters

In [ ]:
model = ElasticNet()
gs = GridSearchCV(estimator = model,
                  param_grid = param_grid,
                  n_jobs = -1,
                  verbose = 2)

gs.fit(X_train, y_train)

In [ ]:
print('Best R2 score:   ', round(gs.best_score_, 4))
print('Best parameters: ', gs.best_params_)

# Define the parameter grid used to tune the decision-tree model

In [ ]:
param_grid = {'max_depth': [5, 10, 20],
              'min_samples_split': [10, 100, 1000],
              'min_samples_leaf': [10, 100, 1000]}

print(param_grid)

# Perform a grid search for optimal decision-tree hyperparameters

In [ ]:
model = DecisionTreeRegressor()
gs = GridSearchCV(estimator = model,
                  param_grid = param_grid,
                  n_jobs = -1,
                  verbose = 2)

gs.fit(X_train, y_train)

In [ ]:
print('Best R2 score:   ', round(gs.best_score_, 4))
print('Best parameters: ', gs.best_params_)

# Compare evaluation metrics for each model

In [ ]:
models = ['Linear Regression', 'Decision Tree',
          'Random Forest', 'XGBoost', 'Dummy Regressor']

metrics = ['R2', 'MAE', 'MSE']

pred_list = ['linreg_y_pred', 'reg_tree_y_pred',
             'rf_y_pred', 'xgb_y_pred', 'dummy_y_pred']

# Baseline algorithm.
dummy = DummyRegressor()
dummy.fit(X_train, y_train)
dummy_y_pred = dummy.predict(X_test)

scores = np.empty((0, 3))

for i in pred_list:
    scores = np.append(scores,
                       np.array([[r2_score(y_test, globals()[i]),
                                  mean_absolute_error(y_test, globals()[i]),
                                  mean_squared_error(y_test, globals()[i])]]),
                       axis = 0)

scores = np.around(scores, 4)

scoring_df = pd.DataFrame(scores, index = models, columns = metrics)
scoring_df.sort_values(by = 'MSE', ascending = True)

# Plot the residuals

In [ ]:
# Set up data frame for plotting.

resid_df = pd.DataFrame()
resid_df['total_amount_usd'] = y_test
resid_df['total_pred'] = xgb_y_pred
resid_df['residuals'] = resid_df['total_amount_usd'] - resid_df['total_pred']
resid_df = resid_df.sort_values('total_amount_usd')[::20]
resid_df['record_num'] = np.arange(len(resid_df))

In [ ]:
plt.figure(figsize = (12, 8))

plt.plot(resid_df['record_num'], resid_df['total_amount_usd'],
         color = 'red', linewidth = 5)
plt.scatter(resid_df['record_num'], resid_df['total_pred'])

plt.legend(['Actual', 'Predicted'])
plt.title('Residual Plot')
plt.ylabel('Total Amount (USD)')
plt.xlabel('Customer Record Number')
plt.show();

# Generate a feature-importance plot

In [ ]:
def feature_importance_plot(model, X_train, n):
    """Plots feature importance. Only works for ensemble learning."""
    plt.figure(figsize = (8, 5))
    feat_importances = pd.Series(model.feature_importances_,
                                 index = X_train.columns)
    feat_importances.nlargest(n).plot(kind = 'barh')
    plt.title(f'Top {n} Features')
    plt.show()

In [ ]:
feature_importance_plot(xgb, X_train, 5)

# Plot learning curves

In [ ]:
def plot_learning_curves(model, X_train, y_train):
    """Plots learning curves for model validation."""
    plt.figure(figsize = (5, 5))
    train_sizes, train_scores, test_scores = \
    learning_curve(model, X_train, y_train, cv = 5,
                   scoring = 'neg_mean_squared_error',
                   n_jobs = -1,
                   shuffle = True,
                   train_sizes = np.linspace(0.01, 1.0, 5))
        
    # Means of training and test set scores.
    train_mean = np.mean(train_scores, axis = 1)
    test_mean = np.mean(test_scores, axis = 1)

    # Draw lines.
    plt.plot(train_sizes, train_mean, '--',
             color = '#111111', label = 'Training score')
    plt.plot(train_sizes, test_mean,
             color = '#111111', label = 'Cross-validation score')

    # Create plot.
    plt.title('Learning Curves')
    plt.xlabel('Training Set Size')
    plt.ylabel('Negative MSE')
    plt.legend(loc = 'best')
    plt.tight_layout()

    plt.show()

In [ ]:
plot_learning_curves(xgb, X_train, y_train)

# Save the best model

In [ ]:
pickle.dump(xgb, open('xgboost_regressor.pickle', 'wb'))